In [1]:
# from lightweight import lightweight
from search import chat
from topics import topics
# from gpt import gpt

/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt="Who is Lady Gaga?"
r=chat(prompt)
t=topics(r)

Researching: Who is Lady Gaga?
Found 15 total results
After deduplication: 13 results
Fetching additional content from URLs...
Researching: Lady Gaga career achievements albums awards 2025 Mayhem best-selling pop music artist
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Loading model meta-llama/Llama-3.2-3B-Instruct on mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.02s/it]


celebrity_&_pop_culture
music
